In [26]:
# import data packages
import pandas as pd
import numpy as np
import datetime
import time
import glob
import os

In [33]:
#retrieve all csv's from the project folder 
all_files = glob.glob(os.path.join( "*.csv"))

df_list = []
# read in all files in 'all_files':
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df_list.append(df)

,Unnamed: 0,The Democrat,Stormy Sanz,Beast Mode,Lesya,Just Tiger,Sharmey,Kenya Nz,No Nukes Skipper Nz,Kivik,We Salute You,Beam Me Up Chopper
0,0,"[datetime.datetime(2022, 11, 27, 20, 32, 37, 3...","[datetime.datetime(2022, 11, 27, 20, 32, 37, 3...","[datetime.datetime(2022, 11, 27, 20, 32, 37, 3...","[datetime.datetime(2022, 11, 27, 20, 32, 37, 3...","[datetime.datetime(2022, 11, 27, 20, 32, 37, 3...","[datetime.datetime(2022, 11, 27, 20, 32, 37, 3...","[datetime.datetime(2022, 11, 27, 20, 32, 37, 3...","[datetime.datetime(2022, 11, 27, 20, 32, 37, 3...","[datetime.datetime(2022, 11, 27, 20, 32, 37, 3...","[datetime.datetime(2022, 11, 27, 20, 32, 37, 3...","[datetime.datetime(2022, 11, 27, 20, 32, 37, 3..."
1,1,"[datetime.datetime(2022, 11, 27, 20, 32, 38, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 38, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 38, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 38, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 38, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 38, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 38, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 38, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 38, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 38, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 38, 5..."
2,2,"[datetime.datetime(2022, 11, 27, 20, 32, 39, 9...","[datetime.datetime(2022, 11, 27, 20, 32, 39, 9...","[datetime.datetime(2022, 11, 27, 20, 32, 39, 9...","[datetime.datetime(2022, 11, 27, 20, 32, 39, 9...","[datetime.datetime(2022, 11, 27, 20, 32, 39, 9...","[datetime.datetime(2022, 11, 27, 20, 32, 39, 9...","[datetime.datetime(2022, 11, 27, 20, 32, 39, 9...","[datetime.datetime(2022, 11, 27, 20, 32, 39, 9...","[datetime.datetime(2022, 11, 27, 20, 32, 39, 9...","[datetime.datetime(2022, 11, 27, 20, 32, 39, 9...","[datetime.datetime(2022, 11, 27, 20, 32, 39, 9..."
3,3,"[datetime.datetime(2022, 11, 27, 20, 32, 41, 2...","[datetime.datetime(2022, 11, 27, 20, 32, 41, 2...","[datetime.datetime(2022, 11, 27, 20, 32, 41, 2...","[datetime.datetime(2022, 11, 27, 20, 32, 41, 2...","[datetime.datetime(2022, 11, 27, 20, 32, 41, 2...","[datetime.datetime(2022, 11, 27, 20, 32, 41, 2...","[datetime.datetime(2022, 11, 27, 20, 32, 41, 2...","[datetime.datetime(2022, 11, 27, 20, 32, 41, 2...","[datetime.datetime(2022, 11, 27, 20, 32, 41, 2...","[datetime.datetime(2022, 11, 27, 20, 32, 41, 2...","[datetime.datetime(2022, 11, 27, 20, 32, 41, 2..."
4,4,"[datetime.datetime(2022, 11, 27, 20, 32, 42, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 42, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 42, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 42, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 42, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 42, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 42, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 42, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 42, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 42, 5...","[datetime.datetime(2022, 11, 27, 20, 32, 42, 5..."


In [20]:
# Convert dataframe so every column is it's own row
log_long = pd.DataFrame.transpose(log, copy=True)
#rename index
log_long.index.name = 'horse'
#number of horses in dataframe
nrow = len(log_long)
#number of columns in dataframe
ncol = len(log_long.columns)
print(f"df has {nrow} horses and {ncol} columns")

df has 43 horses and 1008 columns


In [24]:
#explode the list values into their own columns
log_long.explode(0)


,0,1,2,3,4,5,6,7,8,9,...,998,999,1000,1001,1002,1003,1004,1005,1006,1007
horse,,,,,,,,,,,,,,,,,,,,,
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,...,998,999,1000,1001,1002,1003,1004,1005,1006,1007
Symi Fortuna,"[51.0, 390.0, 1669536597.1688125]","[51.0, 330.0, 1669536598.3914204]","[51.0, 330.0, 1669536599.5513375]","[51.0, 330.0, 1669536600.7313757]","[51.0, 310.0, 1669536601.9278998]","[51.0, 310.0, 1669536603.106389]","[51.0, 310.0, 1669536604.3218853]","[51.0, 310.0, 1669536605.5185754]","[51.0, 310.0, 1669536606.7389627]","[51.0, 310.0, 1669536607.9121716]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Kaniva Swifty,"[13.0, 24.0, 1669536597.1688125]","[13.0, 21.0, 1669536598.3914204]","[13.0, 21.0, 1669536599.5513375]","[13.0, 21.0, 1669536600.7313757]","[13.0, 22.0, 1669536601.9278998]","[13.0, 23.0, 1669536603.106389]","[13.0, 23.0, 1669536604.3218853]","[13.0, 21.0, 1669536605.5185754]","[13.0, 21.0, 1669536606.7389627]","[13.0, 21.0, 1669536607.9121716]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rothwell Horse,"[9.0, 17.0, 1669536597.1688125]","[9.0, 17.0, 1669536598.3914204]","[9.0, 17.0, 1669536599.5513375]","[9.0, 17.0, 1669536600.7313757]","[9.0, 17.0, 1669536601.9278998]","[9.0, 17.0, 1669536603.106389]","[9.0, 17.0, 1669536604.3218853]","[9.0, 17.0, 1669536605.5185754]","[9.0, 17.0, 1669536606.7389627]","[9.0, 17.0, 1669536607.9121716]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Esmay Nelly,"[9.0, 11.5, 1669536597.1688125]","[9.0, 11.5, 1669536598.3914204]","[9.0, 11.5, 1669536599.5513375]","[9.0, 11.5, 1669536600.7313757]","[9.0, 11.5, 1669536601.9278998]","[9.0, 11.5, 1669536603.106389]","[9.0, 11.5, 1669536604.3218853]","[9.0, 11.0, 1669536605.5185754]","[9.0, 11.0, 1669536606.7389627]","[9.0, 11.0, 1669536607.9121716]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cash Destroyer,"[1.55, 1.86, 1669536597.1688125]","[1.55, 1.85, 1669536598.3914204]","[1.55, 1.85, 1669536599.5513375]","[1.55, 1.85, 1669536600.7313757]","[1.55, 1.85, 1669536601.9278998]","[1.55, 1.86, 1669536603.106389]","[1.55, 1.86, 1669536604.3218853]","[1.55, 1.84, 1669536605.5185754]","[1.55, 1.84, 1669536606.7389627]","[1.55, 1.84, 1669536607.9121716]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Symi Ventura,"[91.0, 830.0, 1669536597.1688125]","[91.0, 830.0, 1669536598.3914204]","[91.0, 830.0, 1669536599.5513375]","[91.0, 830.0, 1669536600.7313757]","[91.0, 820.0, 1669536601.9278998]","[91.0, 820.0, 1669536603.106389]","[91.0, 820.0, 1669536604.3218853]","[91.0, 820.0, 1669536605.5185754]","[91.0, 820.0, 1669536606.7389627]","[91.0, 820.0, 1669536607.9121716]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Shavana,"[8.0, 11.0, 1669536597.1688125]","[8.0, 11.0, 1669536598.3914204]","[8.0, 11.0, 1669536599.5513375]","[8.0, 11.0, 1669536600.7313757]","[8.0, 11.0, 1669536601.9278998]","[8.0, 11.0, 1669536603.106389]","[8.0, 11.0, 1669536604.3218853]","[8.0, 11.0, 1669536605.5185754]","[8.0, 11.0, 1669536606.7389627]","[8.0, 11.0, 1669536607.9121716]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Robell Magic,"[6.0, 8.0, 1669536597.1688125]","[6.0, 8.0, 1669536598.3914204]","[6.0, 7.8, 1669536599.5513375]","[6.0, 7.8, 1669536600.7313757]","[6.0, 7.8, 1669536601.9278998]","[6.0, 7.8, 1669536603.106389]","[6.0, 7.8, 1669536604.3218853]","[6.0, 7.8, 1669536605.5185754]","[6.0, 7.8, 1669536606.7389627]","[6.0, 7.8, 1669536607.9121716]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
